## Preprocessing

In [9]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [10]:

## Optimization 1: Drop several columns with very unbalanced distributions
dropCols = ['EIN','SPECIAL_CONSIDERATIONS']
application_df.drop(columns=dropCols,inplace = True,errors='ignore')

# Determine the number of unique values in each column.
application_df.nunique()
for i in application_df.columns:
    print(application_df[i].value_counts())


NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64
APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725

In [11]:
# Choose a cutoff value and create a list of names to be replaced
val_dict = application_df['NAME'].value_counts().to_dict()
cutoff = application_df['NAME'].value_counts().iloc[9] #200 #528
name_to_replace = [i for i in val_dict if val_dict[i] < cutoff]

# Replace in dataframe
for name in name_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Choose a cutoff value and create a list of application types to be replaced
val_dict = application_df['APPLICATION_TYPE'].value_counts().to_dict()
cutoff = application_df['APPLICATION_TYPE'].value_counts().iloc[9]#1065 #528
application_types_to_replace = [i for i in val_dict if val_dict[i] < cutoff]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")


# Choose a cutoff value and create a list of classifications to be replaced
val_dict = application_df['CLASSIFICATION'].value_counts().to_dict()

# print(application_df['CLASSIFICATION'].value_counts())

cutoff = application_df['CLASSIFICATION'].value_counts().iloc[9]#777 #1883
classifications_to_replace = [i for i in val_dict if val_dict[i] < cutoff]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


In [12]:

# Convert categorical data to numeric with `pd.get_dummies`
application_df_convert = pd.get_dummies(application_df)
# Split our preprocessed data into our features and target arrays
y = application_df_convert['IS_SUCCESSFUL'].values
X = application_df_convert.drop(columns = 'IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
rand_state = 42
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state=rand_state)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [13]:

# Compile, Train and Evaluate the Model
## Optimization 2: Change amount of neurons in each layer
number_input_features = X_train.shape[1]
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 20
hidden_nodes_layer3 = 20

nn = tf.keras.Sequential()

# First hidden layer
## Optimization 3: Add more layers and dropout layers to account for overfitting
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
nn.add(tf.keras.layers.Dropout(0.15))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="elu"))
nn.add(tf.keras.layers.Dropout(0.15))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))





# # Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 15)                900       
                                                                 
 dropout_2 (Dropout)         (None, 15)                0         
                                                                 
 dense_5 (Dense)             (None, 20)                320       
                                                                 
 dropout_3 (Dropout)         (None, 20)                0         
                                                                 
 dense_6 (Dense)             (None, 20)                420       
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 1,661
Trainable params: 1,661
Non-traina

In [14]:

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

epochs = 100
fit_model = nn.fit(X_train_scaled,y_train,epochs=epochs,batch_size=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")




Epoch 1/100
515/515 [==============================] - 1s 876us/step - loss: 0.5831 - accuracy: 0.6965
Epoch 2/100
515/515 [==============================] - 0s 887us/step - loss: 0.5393 - accuracy: 0.7345
Epoch 3/100
515/515 [==============================] - 0s 886us/step - loss: 0.5271 - accuracy: 0.7397
Epoch 4/100
515/515 [==============================] - 0s 878us/step - loss: 0.5243 - accuracy: 0.7416
Epoch 5/100
515/515 [==============================] - 0s 884us/step - loss: 0.5220 - accuracy: 0.7429
Epoch 6/100
515/515 [==============================] - 0s 884us/step - loss: 0.5181 - accuracy: 0.7456
Epoch 7/100
515/515 [==============================] - 0s 881us/step - loss: 0.5157 - accuracy: 0.7477
Epoch 8/100
515/515 [==============================] - 0s 881us/step - loss: 0.5139 - accuracy: 0.7495
Epoch 9/100
515/515 [==============================] - 0s 917us/step - loss: 0.5121 - accuracy: 0.7486
Epoch 10/100
515/515 [==============================] - 0s 889us/step - l

In [15]:
nn.save('AlphabetSoupCharity_Optimization.h5')